In [ ]:
import xml.etree.ElementTree as ET
import json
from itertools import count

def xml_to_json(xml_filename):
    tree = ET.parse(xml_filename)
    root = tree.getroot()

    json_data = {"menu": {"category": []}}

    for category in root.findall('category'):
        # Process category-level data
        category_data = {
            "categoryId": category.get("id"),
            "name": {},  # Will contain multilingual names
            "dish": []
        }

        # Process ALL elements under category (not just name/dish)
        for elem in category:
            if elem.tag == 'name':
                lang = elem.get('lang')
                category_data["name"][lang] = elem.text
            elif elem.tag != 'dish':  # Skip dish (we process it separately)
                # Preserve ALL other category-level fields exactly
                category_data[elem.tag] = elem.text

        # Process dishes
        for dish in category.findall('dish'):
            dish_data = {}

            for elem in dish:
                if elem.tag == 'id':
                    dish_data["dishId"] = elem.text
                elif elem.tag in ['name', 'shortDescription', 'longDescription']:
                    # Handle multilingual fields
                    if elem.tag not in dish_data:
                        dish_data[elem.tag] = {}
                    lang = elem.get('lang')
                    dish_data[elem.tag][lang] = elem.text
                elif elem.tag == 'allergens':
                    # Numbered allergens (1, 2, 3...)
                    dish_data["allergens"] = [{str(i+1): a.text} 
                                           for i, a in enumerate(elem.findall('allergen'))]
                elif elem.tag == 'additives':
                    # Lettered additives (a, b, c...)
                    dish_data["additives"] = [{chr(97+i): ad.text} 
                                           for i, ad in enumerate(elem.findall('additive'))]
                else:
                    # Preserve ALL other fields exactly as-is
                    dish_data[elem.tag] = elem.text

            category_data["dish"].append(dish_data)

        json_data["menu"]["category"].append(category_data)

    # Save JSON
    json_filename = xml_filename.replace('.xml', '.json')
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

    return json_filename

# Example usage
if __name__ == "__main__":
    input_xml = "menu.xml"
    output_json = xml_to_json(input_xml)
    print(f"Conversion complete. JSON saved to: {output_json}")

In [ ]:
import xml.etree.ElementTree as ET
import json
from collections import OrderedDict

def xml_to_json(xml_filename):
    tree = ET.parse(xml_filename)
    root = tree.getroot()

    # We'll use OrderedDict to maintain XML element order
    json_data = OrderedDict([
        ("menu", OrderedDict([
            ("category", []),
        ]))
    ])

    # Initialize mappings
    allergen_mapping = OrderedDict()
    additive_mapping = OrderedDict()

    for category in root.findall('category'):
        category_data = OrderedDict([
            ("categoryId", category.get("id")),
            ("name", OrderedDict()),
            ("dish", [])
        ])

        # Process category elements
        for elem in category:
            if elem.tag == 'name':
                lang = elem.get('lang')
                category_data["name"][lang] = elem.text
            elif elem.tag != 'dish':
                category_data[elem.tag] = elem.text

        # Process dishes
        for dish in category.findall('dish'):
            dish_data = OrderedDict()

            for elem in dish:
                if elem.tag == 'id':
                    dish_data["dishId"] = elem.text
                elif elem.tag in ['name', 'shortDescription', 'longDescription']:
                    if elem.tag not in dish_data:
                        dish_data[elem.tag] = OrderedDict()
                    lang = elem.get('lang')
                    dish_data[elem.tag][lang] = elem.text
                elif elem.tag == 'allergens':
                    # Process allergens with numbering
                    allergen_numbers = []
                    for i, allergen in enumerate(elem.findall('allergen'), start=1):
                        allergen_str = str(i)
                        allergen_numbers.append(allergen_str)
                        if allergen_str not in allergen_mapping:
                            allergen_mapping[allergen_str] = allergen.text
                    dish_data["allergens"] = allergen_numbers
                elif elem.tag == 'additives':
                    # Process additives with letters
                    additive_letters = []
                    for j, additive in enumerate(elem.findall('additive')):
                        letter = chr(97 + j)  # 97 = 'a' in ASCII
                        additive_letters.append(letter)
                        if letter not in additive_mapping:
                            additive_mapping[letter] = additive.text
                    dish_data["additives"] = additive_letters
                else:
                    dish_data[elem.tag] = elem.text

            # Ensure additives exists even if empty
            if "additives" not in dish_data:
                dish_data["additives"] = []

            category_data["dish"].append(dish_data)

        json_data["menu"]["category"].append(category_data)

    # Add mappings to the root level
    if allergen_mapping:
        json_data["allergen_mapping"] = allergen_mapping
    if additive_mapping:
        json_data["additive_mapping"] = additive_mapping

    # Save JSON
    json_filename = xml_filename.replace('.xml', '.json')
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

    return json_filename

if __name__ == "__main__":
    input_xml = "menu.xml"
    output_json = xml_to_json(input_xml)
    print(f"Conversion complete. JSON saved to: {output_json}")

### Function to Translate English Words to Amharic and German
This function uses the Google Translate to translate a given English word into Amharic and German.

In [1]:
from googletrans import Translator

def translate_text(text, target_lang):
    translator = Translator()
    try:
        translation = translator.translate(text, dest=target_lang)
        return translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        return text  # Return original text if translation fails

# Example usage
print("German:", translate_text("Hello, how are you?", "de"))  # German
print("Amharic:", translate_text("Hello, how are you?", "am"))  # Amharic
print("English:", translate_text("Hello, how are you?", "en"))  # English (unchanged)

German: Hallo, wie geht es dir?
Amharic: ሰላም እንደምን አለህ፧
Amharic: ሰላም እንደምን አለህ፧
English: Hello, how are you?
English: Hello, how are you?


In [5]:
from deep_translator import GoogleTranslator

def translate_text(text, target_lang):
    try:
        return GoogleTranslator(source='auto', target=target_lang).translate(text)
    except Exception as e:
        print(f"Error: {e}")
        return text

# Test
print("German:", translate_text("Hello, how are you?. I missed you a lot.", "de"))  # de = German
print("Amharic:", translate_text("Hello, how are you?. I missed you a lot.", "am"))  # am = Amharic

German: Hallo, wie geht es dir?. Ich habe dich sehr vermisst.
Amharic: ሰላም እንደምን አለህ፧። ብዙ ናፍቄሻለሁ.
Amharic: ሰላም እንደምን አለህ፧። ብዙ ናፍቄሻለሁ.
